Steven Jonathan

In [ ]:
import pandas as pd

In [ ]:
fpath = '/content/amazon_alexa.tsv'
df = pd.read_csv(fpath,sep='\t') #read the dataset
df.shape

(3150, 5)

In [ ]:
df.head() #check the dataset

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [ ]:
df = df.drop(columns=['date', 'variation', 'feedback']) #drop unimportant part of the analysis

In [ ]:
columns_titles = ["verified_reviews","rating"]
df=df.reindex(columns=columns_titles) #switch the rating columns with verified reviews, for better analysis
df.head()

,verified_reviews,rating
0,Love my Echo!,5
1,Loved it!,5
2,"Sometimes while playing a game, you can answer...",4
3,I have had a lot of fun with this thing. My 4 ...,5
4,Music,5


In [ ]:
rating = pd.get_dummies(df.rating)
new_df = pd.concat([df, rating], axis=1)
new_df  = new_df.drop(columns='rating')
new_df #one-hot-encoding, and created new dataframe

,verified_reviews,1,2,3,4,5
0,Love my Echo!,0,0,0,0,1
1,Loved it!,0,0,0,0,1
2,"Sometimes while playing a game, you can answer...",0,0,0,1,0
3,I have had a lot of fun with this thing. My 4 ...,0,0,0,0,1
4,Music,0,0,0,0,1
...,...,...,...,...,...,...
3145,"Perfect for kids, adults and everyone in betwe...",0,0,0,0,1
3146,"Listening to music, searching locations, check...",0,0,0,0,1
3147,"I do love these things, i have them running my...",0,0,0,0,1
3148,Only complaint I have is that the sound qualit...,0,0,0,0,1


In [ ]:
verified_reviews = new_df['verified_reviews'].values
label = new_df[[1,2,3,4,5]].values

In [ ]:
from sklearn.model_selection import train_test_split
verified_reviews_train, verified_reviews_test, label_train, label_test = train_test_split(verified_reviews, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(verified_reviews_train) 
tokenizer.fit_on_texts(verified_reviews_test)
 
sekuens_train = tokenizer.texts_to_sequences(verified_reviews_train)
sekuens_test = tokenizer.texts_to_sequences(verified_reviews_test)
 
padded_train = pad_sequences(sekuens_train) 
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
num_epochs = 30
history = model.fit(padded_train, label_train, epochs=num_epochs, validation_data=(padded_test, label_test), callbacks=[callbacks])

Epoch 1/30
79/79 [==============================] - 27s 317ms/step - loss: 1.0099 - accuracy: 0.7187 - val_loss: 0.8678 - val_accuracy: 0.7254
Epoch 2/30
79/79 [==============================] - 25s 311ms/step - loss: 0.7966 - accuracy: 0.7274 - val_loss: 0.7304 - val_accuracy: 0.7397
Epoch 3/30
79/79 [==============================] - 25s 313ms/step - loss: 0.6336 - accuracy: 0.7579 - val_loss: 0.7901 - val_accuracy: 0.7429
Epoch 4/30
79/79 [==============================] - 25s 318ms/step - loss: 0.5461 - accuracy: 0.7825 - val_loss: 0.7365 - val_accuracy: 0.7540
Epoch 5/30
79/79 [==============================] - 25s 319ms/step - loss: 0.4881 - accuracy: 0.8091 - val_loss: 0.7602 - val_accuracy: 0.7651
Epoch 6/30
79/79 [==============================] - 25s 317ms/step - loss: 0.4175 - accuracy: 0.8373 - val_loss: 0.8343 - val_accuracy: 0.7476
Epoch 7/30
79/79 [==============================] - 25s 316ms/step - loss: 0.3700 - accuracy: 0.8504 - val_loss: 0.9501 - val_accuracy: 0.7444